In [ ]:
from oiii_fit_copy import OIII
from fit_routine_oiii import WLAX, Lines, lines, oiii_doublet, c, z, chisq
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np
from numpy.typing import NDArray
from scipy.optimize import curve_fit
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
hdul1 = fits.open("ADP.2016-06-01T15_26_54.597.fits")
head = hdul1[1].header
cubehdu = hdul1[1]
cubehead = cubehdu.header
cube = cubehdu.data


varc = hdul1[2].data

hdul2 = fits.open("continuum spectrum.fits")
basespec = hdul2[0].data

oiii = OIII('OIII', lines['OIII'][0], lines['OIII'][1], lines['OIII'][2], cube, varc, basespec)
plt.figure(figsize=(15,6))
plt.step(oiii.wlax, oiii.basespec, where='mid')
oiii.lranges

In [ ]:
quickrej = 0
snrrej = 0
runerr = 0

obs_0 = oiii.obs[0]
mask = (oiii.lranges[0] < oiii.wlax) & (oiii.wlax < oiii.lranges[1])
l_wlax = oiii.wlax[mask]
l_lranges = oiii.lranges

stime = time.time()
for i in tqdm(range(oiii.cube_x), smoothing=1):
    for j in range(oiii.cube_y):
        
        fit_spec, err_spec = oiii.get_fit_spaxel(i, j)

        if type(fit_spec) == type(None):
            quickrej +=1
            oiii.rejcube[0,i,j] = 1
            continue

        try:
            
            popt, pcov = curve_fit(oiii_doublet, l_wlax, fit_spec[mask], p0=[500, obs_0, 2.3, 0], 
                                   bounds=([0,l_lranges[0],0,-50], [2e4, l_lranges[1], 30, 50]), 
                                   absolute_sigma=True, sigma=err_spec[mask])

            uncertainty = np.sqrt(np.diagonal(pcov))
            snr = (popt[0]+popt[1])/np.sqrt(uncertainty[0]**2+uncertainty[1]**2)
            chi = chisq(oiii_doublet, l_wlax, fit_spec[mask], err_spec[mask], popt)
            
            
            if snr > 3.:
                oiii.fitcube[:4,i,j] = popt[:]
                oiii.fitcube[4,i,j] = snr
                oiii.fitcube[5,i,j] = chi
                oiii.fiterrcube[:,i,j] = uncertainty
            else:
                oiii.fitcube[:4,i,j] = np.nan
                oiii.fitcube[4,i,j] = snr
                oiii.fitcube[5,i,j] = chi
                  
                
                oiii.rejcube[1,i,j] = 1
                snrrej += 1
                
        except (RuntimeError, ValueError):
            oiii.set_to_nan(i, j)
            oiii.rejcube[2,i,j] = 1
            runerr +=1

print(quickrej)
print(snrrej)
print(runerr)
print(time.time() - stime)


from astropy.wcs import WCS

newwcs = WCS(cubehead, naxis=2)
newhead = newwcs.to_header()
prihdu = fits.PrimaryHDU(oiii.fitcube[0], header=newhead)
newoiiihdus = [fits.ImageHDU(oiii.fitcube[i]) for i in range(1,oiii.fitcube.shape[0])]
oiiierrhdus = [fits.ImageHDU(oiii.fiterrcube[i]) for i in range(oiii.fiterrcube.shape[0])]
hdul = fits.HDUList([prihdu] + newoiiihdus + oiiierrhdus)
hdul.writeto('oiii_fit_cont.fits', overwrite = True)

rejhdus = fits.PrimaryHDU(oiii.rejcube[0], header=newhead)
otherrejhdus = [fits.ImageHDU(oiii.rejcube[i]) for i in range(1,oiii.rejcube.shape[0])]
hdul2 = fits.HDUList([rejhdus]+otherrejhdus)
hdul2.writeto('oiii_rej_cont.fits', overwrite = True)

detectedimg = np.nan_to_num(oiii.fitcube[0])
snrrejimg = np.nan_to_num(oiii.rejcube[1])
evalimg = detectedimg + snrrejimg

evalhdus = fits.PrimaryHDU(evalimg, header=newhead)
hdul3 = fits.HDUList([evalhdus])
hdul3.writeto('oiii_eval_cont.fits', overwrite = True)

flux0hdus = [prihdu]
velhdus = [fits.ImageHDU(c*(oiii.fitcube[1]/oiii.rest[0]-1-z))]
vdisphdus = [fits.ImageHDU(c*(oiii.fitcube[2]/oiii.rest[0]))]
resulthdus =  fits.HDUList(flux0hdus+velhdus+vdisphdus)
resulthdus.writeto('oiii_results_cont.fits', overwrite=True)

snrhdus = fits.PrimaryHDU(oiii.fitcube[4])
chihdus = [fits.ImageHDU(oiii.fitcube[5])]
snrandchihdus = fits.HDUList([snrhdus]+chihdus)
snrandchihdus.writeto('oiii_snrandchi_cont.fits', overwrite=True)
